In [19]:
import pandas as pd
from utils import umap_func,extract_projections,umap_2d_scatter_chart,create_folder, get_embedding
import os
from transformers import AutoTokenizer
from transformers import AutoModel
import joblib
pd.set_option('display.max_columns', None)


# The notebook create an embeddings representation of the text rule and show it a interactive chart
### It uses the trasformers library instead of the sentence-transformers one 

In [20]:
df=pd.read_csv("out/1_elaborated_dataset.csv")
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button)
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice)
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.turn_off_all_hue', 'name...",2,WeMo Light Switch,Philips Hue,Lighting,Lighting,['Ambient Luminance'],Other Trigger
2556,9959,False,"[{'name': 'RSS Feed', 'module_name': 'feed', '...","When Adafruit releases a new product, blink!",Ktup6MV2-when-adafruit-releases-a-new-product-...,Ktup6MV2,10.0,"When Adafruit releases a new product, blink!",[{'id': '/triggers/feed.new_feed_item_matches'...,F

## Embed "name" feature through language model
use some embedding models like the all-mpnet-base-v2 model and ModernBERT-base

In [21]:

model_embedding_name="answerdotai/ModernBERT-base"#"sentence-transformers/all-mpnet-base-v2"#"BAAI/bge-large-en-v1.5"#"bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_embedding_name)
model = AutoModel.from_pretrained(model_embedding_name)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

c:\Users\nicleonard\AppData\Local\anaconda3\envs\hiis\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning:

`huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicleonard\.cache\huggingface\hub\models--answerdotai--ModernBERT-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development



tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [22]:
print(model)

print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

ModernBertModel(
  (embeddings): ModernBertEmbeddings(
    (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
    (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (layers): ModuleList(
    (0): ModernBertEncoderLayer(
      (attn_norm): Identity()
      (attn): ModernBertAttention(
        (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
        (rotary_emb): ModernBertRotaryEmbedding()
        (Wo): Linear(in_features=768, out_features=768, bias=False)
        (out_drop): Identity()
      )
      (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): ModernBertMLP(
        (Wi): Linear(in_features=768, out_features=2304, bias=False)
        (act): GELUActivation()
        (drop): Dropout(p=0.0, inplace=False)
        (Wo): Linear(in_features=1152, out_features=768, bias=False)
      )
    )
    (1-21): 21 x ModernBertEncoderLayer(
      (attn_norm): LayerNorm((768,), eps=1e-05, e

## Use the "name" column attribute as the only textual representation of the rule

In [23]:
list(df["name"])

['Turn on/off your lights with one tap on your phone',
 'Automatically turn your lights on at sunset',
 'Turn your lights on automatically as you arrive home',
 'Tell Alexa to start a Harmony activity',
 'Blink your Hue lights when your Amazon Alexa timer hits 0',
 'Tell Alexa to adjust your Nest Thermostat to a certain temperature',
 'Set your Nest thermostat to a certain temperature with Button widget',
 'Tell Alexa to start the party with a Hue light color show',
 'Tell Alexa to start the party and put your lights on a color loop',
 'Park mower if weather forecast predicts frost or heavy rain',
 'Tell Google Assistant to set the scene',
 '"Alexa, trigger self destruct"',
 'Turn on the lights when you arrive home',
 'Automatically turn your Hue lights blue whenever it starts to rain',
 'Toggle Yeelight on/off',
 'Dim down the lights',
 'Set your Nest thermostat to ___°',
 'Close Car Garage Door',
 'Set the temperature on your Nest Thermostat when you are close to home',
 'Start a par

In [24]:
embeddings = get_embedding(list(df["name"]), tokenizer, model)

In [25]:
embeddings.shape,embeddings# some checks

((2560, 768),
 array([[ 1.4016649 , -1.0469215 ,  0.06637527, ..., -0.6114791 ,
         -0.06263749, -0.86379087],
        [ 0.9716516 , -0.8896024 ,  0.07050248, ..., -0.4613184 ,
          0.01163797, -0.6540833 ],
        [ 1.0459199 , -0.5189052 , -0.07820141, ..., -0.28311902,
          0.1928068 , -0.63038075],
        ...,
        [ 0.8028022 , -0.56863177, -0.35809335, ..., -0.50351375,
         -0.09572634, -0.3810719 ],
        [ 0.90647936, -0.49675092, -0.12605315, ..., -0.36970887,
          0.00744445, -0.5536981 ],
        [ 0.73101306, -0.308505  , -0.12493306, ..., -0.52691317,
          0.18261479, -0.38305348]], dtype=float32))

In [26]:
df["embeddings"]=list(embeddings)

In [27]:
#take the last part for the name
model_embedding_name=model_embedding_name.split("/")[-1]

In [28]:
#save the calculated embeddings
emdeddings_path="out/embeddings_"+model_embedding_name
joblib.dump(embeddings, emdeddings_path)

['out/embeddings_ModernBERT-base']

In [29]:
# just a check
emdeddings_path="out/embeddings_"+model_embedding_name
embeddings = joblib.load(emdeddings_path)
embeddings,len(embeddings),embeddings[0].shape

(array([[ 1.4016649 , -1.0469215 ,  0.06637527, ..., -0.6114791 ,
         -0.06263749, -0.86379087],
        [ 0.9716516 , -0.8896024 ,  0.07050248, ..., -0.4613184 ,
          0.01163797, -0.6540833 ],
        [ 1.0459199 , -0.5189052 , -0.07820141, ..., -0.28311902,
          0.1928068 , -0.63038075],
        ...,
        [ 0.8028022 , -0.56863177, -0.35809335, ..., -0.50351375,
         -0.09572634, -0.3810719 ],
        [ 0.90647936, -0.49675092, -0.12605315, ..., -0.36970887,
          0.00744445, -0.5536981 ],
        [ 0.73101306, -0.308505  , -0.12493306, ..., -0.52691317,
          0.18261479, -0.38305348]], dtype=float32),
 2560,
 (768,))

## Create a visual representation 

In [30]:
#create a 2d umap projection
#https://umap-learn.readthedocs.io/en/latest/
projections_2d=umap_func(embeddings, n_components=2)
projections_2d

c:\Users\nicleonard\AppData\Local\anaconda3\envs\hiis\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



generating projections with UMAP took: 5.47 sec


array([[ 3.4788322,  7.418632 ],
       [ 4.68335  ,  6.122862 ],
       [ 6.937518 ,  9.723984 ],
       ...,
       [ 2.588495 ,  9.3411875],
       [ 4.527659 , 10.749243 ],
       [ 6.2383094, 11.411163 ]], dtype=float32)

In [31]:
first_projection, second_projection=extract_projections(projections_2d)
#first_projection, second_projection

In [32]:
df["umap_2d_first"]=first_projection
df["umap_2d_second"]=second_projection

In [33]:
df

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),"[1.4016649, -1.0469215, 0.06637527, -0.2135758...",3.478832,7.418632
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,"[0.9716516, -0.8896024, 0.07050248, -0.1931493...",4.683350,6.122862
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,"[1.0459199, -0.5189052, -0.07820141, -0.097427...",6.937518,9.723984
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),"[0.77145404, -0.5155644, -0.09712972, -0.02735...",6.236907,10.710479
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,"[0.82812214, -0.59726536, -0.13227865, -0.0203...",7.998817,10.831709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/

In [34]:
#create and save a interactive scatter chart (saved into the out_folder)
umap_2d_scatter_chart(df,model_embedding_name)

### example from bert_base_uncased
<img src="figures/bert_uncased_umap_2d.png" width=1000>

From the representation (a 2d proxy of the embeddings space) it is clear how points close in the space (semantically similar from the point of view of the textual representation) have different labels associated with them (colors). The ideal would be for points close to each other to have the same labels. 
Se want to reach this result at the end of our experimentation 

## Save the elaborated dataframe

In [15]:
#create_folder(os.getcwd(),next_path="out")

In [35]:
df.to_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv", index=False)

In [36]:
# just a check
pd.read_csv("out/2_elaborated_dataset_with_embeddings_"+model_embedding_name+"_.csv")

,Unnamed: 0,by_service_owner,channels,description,friendly_id,id,installs_count,name,permissions,pro_features,requires_android_app,requires_ios_app,requires_mobile_app,service_names,service_slug,services,speed,uniq_permissions,services_len,service_triggers,service_actions,triggers_category,actions_category,goal,operation,embeddings,umap_2d_first,umap_2d_second
0,6913,True,"[{'name': 'Button widget', 'module_name': 'do_...",NaN,wYf8ceZQ-turn-on-off-your-lights-with-one-tap-...,wYf8ceZQ,164055.0,Turn on/off your lights with one tap on your p...,[{'id': '/triggers/do_button.do_button_new_com...,False,False,False,False,"['Button widget', 'Philips Hue']",hue,"['do_button', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/do_button.do_button_new_com...,2,Button widget,Philips Hue,Mobile devices & accessories,Lighting,['Ambient Luminance'],Explicit Control (Button),[ 1.40166485e+00 -1.04692149e+00 6.63752705e-...,3.478832,7.418632
1,11608,True,"[{'name': 'Weather Underground', 'module_name'...",Never be left in the dark. Whenever the sun st...,PVkgiLYy-automatically-turn-your-lights-on-at-...,PVkgiLYy,144011.0,Automatically turn your lights on at sunset,"[{'id': '/triggers/weather.sun_sets', 'name': ...",False,False,False,False,"['Weather Underground', 'Philips Hue']",hue,"['weather', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/triggers/weather.sun_sets', 'name': ...",2,Weather Underground,Philips Hue,Weather,Lighting,['Ambient Luminance'],Time Trigger,[ 9.71651614e-01 -8.89602423e-01 7.05024824e-...,4.683350,6.122862
2,27384,True,"[{'name': 'Location', 'module_name': 'location...","Center the map on your home's location, and yo...",HTak4X5f-turn-your-lights-on-automatically-as-...,HTak4X5f,69594.0,Turn your lights on automatically as you arriv...,[{'id': '/triggers/location.enter_region_locat...,False,False,False,True,"['Location', 'Philips Hue']",hue,"['location', 'hue']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/location.enter_region_locat...,2,Location,Philips Hue,Location,Lighting,['Ambient Luminance'],Spatial Trigger,[ 1.04591990e+00 -5.18905222e-01 -7.82014132e-...,6.937518,9.723984
3,2745,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Choose a phrase to control your Harmony Home w...,sHYgpAeS-tell-alexa-to-start-a-harmony-activity,sHYgpAeS,66448.0,Tell Alexa to start a Harmony activity,[{'id': '/triggers/amazon_alexa.custom_utteran...,False,False,False,False,"['Amazon Alexa', 'Harmony']",harmony,"['amazon_alexa', 'harmony']",Realtime Applets usually run within 10 seconds,[{'id': '/triggers/amazon_alexa.custom_utteran...,2,Amazon Alexa,Harmony,Voice assistants,Smart hubs & systems,['Control Hub'],Explicit Control (Voice),[ 7.71454036e-01 -5.15564382e-01 -9.71297175e-...,6.236907,10.710479
4,27491,True,"[{'name': 'Amazon Alexa', 'module_name': 'amaz...",Add some visual flair to your countdown. Tip: ...,funBgceh-blink-your-hue-lights-when-your-amazo...,funBgceh,47628.0,Blink your Hue lights when your Amazon Alexa t...,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",False,False,False,False,"['Amazon Alexa', 'Philips Hue']",hue,"['amazon_alexa', 'hue']",Realtime Applets usually run within 10 seconds,"[{'id': '/triggers/amazon_alexa.timer_fired', ...",2,Amazon Alexa,Philips Hue,Voice assistants,Lighting,['Alert User'],Other Trigger,[ 8.28122139e-01 -5.97265363e-01 -1.32278651e-...,7.998817,10.831709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,13693,False,"[{'name': 'WeMo Light Switch', 'module_name': ...",Long press Wemo Switch = All Hue lights off,qPhRymsC-long-press-wemo-switch-all-hue-lights...,qPhRymsC,10.0,Long press Wemo Switch = All Hue lights off,[{'id': '/triggers/wemo_light_switch.attribute...,False,False,False,False,"['WeMo Light Switch', 'Philips Hue']",wemo_light_switch,"['wemo_light_switch', 'hue']",Polling Applets usually run within 1 hour,"[{'id': '/actions/hue.